In [ ]:
def prGreen(skk): print("\033[92m{}\033[00m" .format(skk))
def prRed(skk): print("\033[91m {}\033[00m" .format(skk))

In [ ]:
def parseGrammar(text):
    index=0
    grammar={}
    head=None
    tokens=[]
    for i in text.split('\n'):
        tokens+=i.split(' ')
    while(index<len(tokens)):
        if(index +1 < len(tokens) and tokens[index+1]=='->'):
            head=tokens[index]
            index+=2
            grammar[head]=[[]]
        elif(tokens[index]=='|'):
            grammar[head].append([])
            index+=1
        else:
            grammar[head][len(grammar[head])-1].append(tokens[index])
            index+=1
    return grammar

In [ ]:
def leftFactoring(grammar):
  keys=list(grammar.keys())
  for i in range(0,len(keys)):
    helperName=keys[i]+"'"
    hasPrefix=True
    while(hasPrefix):
      hasPrefix=False
      longest=[]
      gen=grammar[keys[i]]
      for j in range(0,len(gen)):
        for k in range(j+1,len(gen)):
          l=0
          while(l<len(gen[j]) and l<len(gen[k])):
            if(gen[j][l] != gen[k][l]):
              break
            l+=1
          if(l>0):
            hasPrefix=True
            if(l>len(longest)):
              longest.append(gen[j][0])
      if(hasPrefix):
        while(helperName in grammar):
          helperName+="'"
        grammar[helperName]=[]
        j=0
        for k in range(0,len(gen)):
          if(len(gen[k])>=len(longest) and gen[k][0:len(longest)]==longest):
            if(len(gen[k])==len(longest)):
              grammar[helperName].append(['ϵ'])
            else:
              grammar[helperName].append(gen[k][len(longest):])
          else:
            grammar[keys[i]][j]=grammar[keys[i]][k]
            j+=1
        longest.append(helperName)
        grammar[keys[i]]=[longest]+grammar[keys[i]][0:j]
  return grammar

In [ ]:
def leftRecursion(grammar):
  keys=list(grammar.keys())
  for i in range(len(keys)):
    for j in range(0,i):
      extended=[]
      for k in range(0,len(grammar[keys[i]])):
        if(len(grammar[keys[i]][k])>0 and grammar[keys[i]][k][0]==keys[j]):
          for l in range(len(grammar[keys[j]])):
            extended.append(grammar[keys[j]][l] + grammar[keys[i]][k][1:])
        elif(len(grammar[keys[i]][k])>0):
          extended.append(grammar[keys[i]][k])
      grammar[keys[i]]=extended
    hasDirectRec=False
    for k in range(len(grammar[keys[i]])):
      if(len(grammar[keys[i]][k])>0 and grammar[keys[i]][k][0]==keys[i]):
        hasDirectRec=True
        break
    if(hasDirectRec):
      helperName=keys[i]+"'"
      while(helperName in grammar):
        helperName+="'"
      grammar[helperName]=[]
      j=0
      for k in range(len(grammar[keys[i]])):
        if(len(grammar[keys[i]][k])>0):
          if(grammar[keys[i]][k][0]==keys[i]):
            grammar[helperName].append(grammar[keys[i]][k][1:] + [helperName])
          else:
            if(len(grammar[keys[i]][k])==1 and grammar[keys[i]][k][0]=='ϵ'):
              grammar[keys[i]][k]=[helperName]
            else:
              grammar[keys[i]][k].append(helperName)
            grammar[keys[i]][j]=grammar[keys[i]][k]
            j+=1
      grammar[keys[i]]=grammar[keys[i]][0:j]
      grammar[helperName].append(['ϵ'])
  try:
    x=helperName[:1]
    if(len(grammar[x])==0):
      grammar[x]=[[helperName]]
  except:
    pass
  return grammar    


In [ ]:
def toPretty(grammar):
  pretty=''
  left=0
  keys=list(grammar.keys())
  for i in range(len(keys)):
    for j in range(len(grammar[keys[i]])):
      if(j==0):
        pretty+=keys[i]
        pretty+=' -> '
      else:
        pretty=pretty[0:-1]
        pretty+=' | '
      pretty+=' '.join(grammar[keys[i]][j])+'\n'
  return pretty

In [ ]:
def calcnullables(grammar):
  nullables={}
  keys=list(grammar.keys())
  for i in range(0,len(keys)):
    for j in range(0,len(grammar[keys[i]])):
      for k in range(0,len(grammar[keys[i]][j])):
        if(not grammar[keys[i]][j][k] in grammar):
          nullables[grammar[keys[i]][j][k]]=False
  if('ϵ' in nullables):
    nullables['ϵ']=True
  def calcrec(key,path):
    try:
      if(path.index(key)>=0):
        return false
    except:
      pass
    path.append(key)
    if(key in nullables):
      return nullables[key]
    for ii in range(0,len(grammar[key])):
      jj=0
      while(jj<len(grammar[key][ii])):
        if(not calcrec(grammar[key][ii][jj],path)):
          x=path.pop()
          break
        jj+=1
      if(jj==len(grammar[key][ii])):
        nullables[key]=True
        return True
    nullables[key]=False
    return False
  for i in keys:
    calcrec(i,[])
  return nullables


In [ ]:
def calcFirsts(grammar,nullables):
  firsts={}
  finished=False
  keys=list(grammar.keys())
  for i in range(len(keys)):
    for j in range(len(grammar[keys[i]])):
      for k in range(len(grammar[keys[i]][j])):
        if(not grammar[keys[i]][j][k] in grammar):
          firsts[grammar[keys[i]][j][k]]=[grammar[keys[i]][j][k]]
  def calcRec(key,path):
    if(not key in grammar):
      return firsts[key]
    try:
      if(path.index(key)>=0):
        return firsts[key]
    except:
      pass
    path.append(key)
    if(not key in firsts):
      firsts[key]=[]
      finished=False
    for ii in range(len(grammar[key])):
      jj=0
      while(jj<len(grammar[key][ii])):
        first=calcRec(grammar[key][ii][jj],path)
        for kk in range(len(first)):
          if(firsts[key].count(first[kk])==0 and first[kk]!='ϵ'):
            firsts[key].append(first[kk])
            finished=False
        if(not nullables[grammar[key][ii][jj]]):
          break
        jj+=1
      if(jj==len(grammar[key][ii])):
        if(firsts[key].count('ϵ')==0):
          firsts[key].append('ϵ')
    return firsts[key]
  while(not finished):
    finished=True
    for i in range(len(keys)):
      calcRec(keys[i],[])
  for i in list(firsts.keys()):
    firsts[i].sort()
  return firsts


In [ ]:
def calcFollows(grammar,nullables,firsts):
  follows={}
  keys=list(grammar.keys())
  for i in range(len(keys)):
    if(i==0):
      follows[keys[i]]=['$']
    else:
      follows[keys[i]]=[]
  finished=[False]
  def calc(key):
    for ii in range(len(grammar[key])):
      for jj in range(len(grammar[key][ii])):
        mid=grammar[key][ii][jj]
        if(mid in grammar):
          kk=jj+1
          while(kk<len(grammar[key][ii])):
            first=firsts[grammar[key][ii][kk]]
            for ll in range(len(first)):
              if(first[ll]!='ϵ'):
                try:
                  x=follows[mid].index(first[ll])
                except:
                  follows[mid].append(first[ll])
                  finished[0]=False
            if(not nullables[grammar[key][ii][kk]]):
              break
            kk+=1
          if(kk==len(grammar[key][ii])):
            for ll in range(0,len(follows[key])):
              try:
                x=follows[mid].index(follows[key][ll])
              except:
                follows[mid].append(follows[key][ll])
                finished[0]=False

  while(finished[0]==False):
    finished[0]=True
    for i in range(len(keys)):
      calc(keys[i])
  for i in keys:
    follows[i].sort()
  return follows


In [ ]:
def ll1(grammar,nullables,firsts,follows):
  table={}
  keys=list(grammar.keys())
  for i in range(len(keys)):
    table[keys[i]]={}
    for j in range(len(grammar[keys[i]])):
      k=0
      while(k<len(grammar[keys[i]][j])):
        first=firsts[grammar[keys[i]][j][k]]
        for l in range(len(first)):
          if(first[l]!='ϵ'):
            if(not first[l] in table[keys[i]]):
              table[keys[i]][first[l]]=[]
            table[keys[i]][first[l]].append(grammar[keys[i]][j])
        if(not nullables[grammar[keys[i]][j][k]]):
          break
        k+=1
      if(k==len(grammar[keys[i]][j])):
        follow=follows[keys[i]]
        for l in range(len(follow)):
          if(not follow[l] in table[keys[i]]):
            table[keys[i]][follow[l]]=[]
          table[keys[i]][follow[l]].append(grammar[keys[i]][j])
  key1=list(table.keys())
  for i in key1:
    key2=list(table[i].keys())
    for j in key2:
      if(table[i][j].count(table[i][j][0])>1):
        temp=table[i][j].pop()
  return table
  

In [ ]:
from prettytable import PrettyTable

def showll1(grammar,table,nullables):
  keys=list(grammar.keys())
  symbol=[]
  symbols=list(nullables.keys())
  for i in keys:
    symbols.remove(i)
  if('ϵ' in symbols):
    symbols.remove('ϵ')
  symbols.sort()
  symbols.append('$')
  symbols.insert(0,' ')
  table_header=PrettyTable(symbols)
  table_contents=[]
  key1=list(table.keys())
  fin=1
  for i in key1:
    table_contents.append([i])
    temp=[' ']*(len(symbols)-1)
    table_contents[-1]=table_contents[-1]+temp
    key2=list(table[i])
    for j in key2:
      if(len(table[i][j])>1):
        if(fin==1):
          prRed('This grammar is not suitable for ll1 parsing')
          fin=0
        table_contents[-1][symbols.index(j)]=[]
        for k in range(len(table[i][j])):
          table_contents[-1][symbols.index(j)].append(table_contents[-1][0]+' -> '+' '.join(table[i][j][k]))
        break
      else:
        table_contents[-1][symbols.index(j)]=table_contents[-1][0]+' -> '+' '.join(table[i][j][0])
  for ii in table_contents:
    table_header.add_row(ii)
  print(table_header)


In [ ]:
def parse_string():  
  parse=input('Enter a string: ')
  stack=['$',text[0]]
  input1=parse.split()
  input1.append('$')
  flag=0
  parse_header=PrettyTable(['Rule','Stack','Input'])
  parse_contents=[[' ',' '.join(stack),' '.join(input1)]]
  while (stack[-1]!='$' or input1[0]!='$'):
    top=stack[-1]
    var=input1[0]
    if(top==var):
      temp=stack.pop()
      temp=input1.pop(0)
    else:
      try:
        rule=p7[top][var]
      except:
        flag=1
        break
      temp=stack.pop()
      stack+=rule[0][::-1]
    if(stack[-1]=='ϵ'):
      x=stack.pop()
    parse_contents.append([])
    if(top in p3):
      parse_contents[-1].append(top+' -> '+' '.join(rule[0]))
    else:
      parse_contents[-1].append('match pop')
    parse_contents[-1].append(' '.join(stack))
    parse_contents[-1].append(' '.join(input1))
  for i in parse_contents:
    parse_header.add_row(i)
  if (flag==1):
    parse_contents.append(['**********',"\033[91m{}\033[00m" .format('String not accepted'),'**********'])
    parse_header.add_row(parse_contents[-1])
  else:
    parse_contents.append(['***********',"\033[92m{}\033[00m" .format('String accepted'),'**********'])
    parse_header.add_row(parse_contents[-1])
  print(parse_header)

In [ ]:
text=''
rule='1'
prGreen("Enter Grammar:( copy ϵ if you require lambda)\nEnter $ to stop")
while(rule!='$'):
  rule=input().strip()
  if(rule=='$'):
    break
  text+='\n'+rule
text=text[1:]
text=f'G -> {text[0]}'+'\n'+text
prGreen("Entered grammar after Augmentation: ")
print(text+'\n')


Enter Grammar:( copy ϵ if you require lambda)
Enter $ to stop
S -> a S b S | b S a S | ϵ
$
Entered grammar after Augmentation: 
G -> S
S -> a S b S | b S a S | ϵ



In [ ]:
prGreen('After Parsing:')
p1=parseGrammar(text)
for i in p1:
  print(i+" :"+str(p1[i]))
prGreen('\nAfter left Factoring:')
p2=leftFactoring(p1)
print(toPretty(p2))
prGreen('After left recursion removal:')
p3=leftRecursion(p2)
print(toPretty(p3))
p4=calcnullables(p3)
prGreen('Nullable items:')
for i in p4:
  print(i,':',p4[i])
p5=calcFirsts(p3,p4)
prGreen('\nFirsts:')
for i in p5:
  print(i,':',', '.join(p5[i]))
p6=calcFollows(p3,p4,p5)
prGreen('\nFollows:')
for i in p6:
  print(i,':',', '.join(p6[i]))
p7=ll1(p3,p4,p5,p6)
print(p7)
prGreen('\nll1 table')
showll1(p3,p7,p4)
parse_string()

After Parsing:
G :[['S']]
S :[['a', 'S', 'b', 'S'], ['b', 'S', 'a', 'S'], ['ϵ']]

After left Factoring:
G -> S
S -> a S b S | b S a S | ϵ

After left recursion removal:
G -> S
S -> a S b S | b S a S | ϵ

Nullable items:
a : False
b : False
ϵ : True
S : True
G : True

Firsts:
a : a
b : b
ϵ : ϵ
G : a, b, ϵ
S : a, b, ϵ

Follows:
G : $
S : $, a, b
{'G': {'a': [['S']], 'b': [['S']], '$': [['S']]}, 'S': {'a': [['a', 'S', 'b', 'S'], ['ϵ']], 'b': [['b', 'S', 'a', 'S'], ['ϵ']], '$': [['ϵ']]}}

ll1 table
 This grammar is not suitable for ll1 parsing
+---+----------------------------+--------+--------+
|   |             a              |   b    |   $    |
+---+----------------------------+--------+--------+
| G |           G -> S           | G -> S | G -> S |
| S | ['S -> a S b S', 'S -> ϵ'] |        |        |
+---+----------------------------+--------+--------+
Enter a string: abab
+------------+---------------------+------------+
|    Rule    |        Stack        |   Input    |
+------------+-